<h2 style="text-align:center;">
<b>COMPREHENSIVE RETAIL ANALYTICS THROUGH PRODUCT AND CUSTOMER SEGMENTATION</b>
</h2>

<h4 style="text-align:center;">
<i>Author: Isaiah Akuku</i>
</h4>


In [2]:
#Check version
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.20.0
ipykernel        : 6.28.0
ipywidgets       : not installed
jupyter_client   : 8.6.0
jupyter_core     : 5.5.0
jupyter_server   : 2.10.0
jupyterlab       : 4.0.13
nbclient         : 0.8.0
nbconvert        : 7.10.0
nbformat         : 5.9.2
notebook         : 7.0.8
qtconsole        : not installed
traitlets        : 5.7.1


In [3]:
#pip install pickleshare
#pip install pygments
#pip install python-docx
#pip install pandas matplotlib seaborn python-docx
#pip install notebook
#pip install wordcloud

In [4]:
#Set directory
#%cd "C:/Users/ADMIN/Documents/DSA/Spring 2025/DSA2040A/Assignment/Group Project/Restaurant Data"
%cd "C:/Users/ADMIN/Documents/DSA/Spring 2025/DSA2040A/Assignment/Group Project"

C:\Users\ADMIN\Documents\DSA\Spring 2025\DSA2040A\Assignment\Group Project


In [5]:
#os.listdir()

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------
# TASK A: Comprehensive retail analytics framework
#----------------------------------------------------------------------------------------------------------------------------
#Task 0. Environment setup
#----------------------------------------------------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from sklearn.metrics import accuracy_score, silhouette_score

import os
import sys
from sklearn.preprocessing import KBinsDiscretizer
from mlxtend.frequent_patterns import fpgrowth, association_rules
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from docx import Document
from docx.shared import Inches

# Configuration
plt.rcParams.update({
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10
})
sns.set_palette("husl")
os.makedirs('04 visualizations', exist_ok=True)

#----------------------------------------------------------------------------------------------------------------------------
# Task 1. Data loading and preparation
#----------------------------------------------------------------------------------------------------------------------------
def load_and_clean_data():
    """Load and clean the retail transaction data"""
    try:
        df = pd.read_excel('retail_dataset.xlsx',
                           usecols=['Invoice', 'StockCode', 'Description',
                                    'Quantity', 'InvoiceDate', 'Price', 'Country', 'Customer ID'],
                           parse_dates=['InvoiceDate'],
                           engine='openpyxl')
    except FileNotFoundError:
        print("Error: Data file not found. Please ensure 'retail_dataset.xlsx' exists.")
        sys.exit(1)

    # Rename columns
    df = df.rename(columns={
        'Invoice': 'InvoiceNo',
        'Price': 'UnitPrice',
        'Customer ID': 'CustomerID'
    })

    print("\n=== Data Cleaning ===")
    print("Initial records:", len(df))

    # Clean product descriptions
    df['Clean_Description'] = df['Description'].str.strip().str.lower()
    mask = df['Clean_Description'].str.contains(r'^\?+.*|\?+.*\?+$|^\?+$', na=False)
    df = df[~mask]
    df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

    # Create categories
    df['Price_Cat'] = pd.cut(df['UnitPrice'], 
                            bins=[-np.inf, 5, 10, 50, np.inf],
                            labels=['Budget', 'Mid-Range', 'Premium', 'Luxury'])
    df['Quantity_Cat'] = pd.cut(df['Quantity'].abs(),
                               bins=[-np.inf, 5, 10, 50, np.inf],
                               labels=['Single', 'Multi', 'Bulk', 'Wholesale'])

    return df.dropna(subset=['Clean_Description']).reset_index(drop=True)

#----------------------------------------------------------------------------------------------------------------------------
# Task 2: Product Clustering
#----------------------------------------------------------------------------------------------------------------------------
def cluster_products(df):
    """Cluster products using TF-IDF and KMeans"""
    print("\n=== Product Clustering ===")
    
    # TF-IDF Vectorization
    tfidf = TfidfVectorizer(max_df=0.85, min_df=5, stop_words='english',
                           ngram_range=(1,2), max_features=5000)
    tfidf_matrix = tfidf.fit_transform(df['Clean_Description'])

    # Cluster optimization
    inertias = []
    silhouette_scores = []
    k_range = range(3, 8)
    
    for k in k_range:
        kmeans = KMeans(n_clusters=k, random_state=2040, n_init=10)
        labels = kmeans.fit_predict(tfidf_matrix)
        inertias.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(tfidf_matrix, labels))

    # Visualization
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(k_range, inertias, marker='o')
    plt.title('Elbow Method')
    plt.subplot(1,2,2)
    plt.plot(k_range, silhouette_scores, marker='o', color='orange')
    plt.title('Silhouette Scores')
    plt.savefig('04 visualizations/clustering_metrics.png', dpi=300)
    plt.close()

    # Final clustering
    optimal_k = 6
    kmeans = KMeans(n_clusters=optimal_k, random_state=2040, n_init=20)
    df['Product_Cluster'] = kmeans.fit_predict(tfidf_matrix)
    
    # Cluster labeling
    cluster_labels = {}
    for cluster_id in range(optimal_k):
        text = ' '.join(df[df['Product_Cluster'] == cluster_id]['Clean_Description'].sample(500, replace=True))
        wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
        plt.figure(figsize=(10,5))
        plt.imshow(wordcloud)
        plt.title(f"Cluster {cluster_id} Keywords")
        plt.axis('off')
        plt.savefig(f'04 visualizations/cluster_{cluster_id}_keywords.png', bbox_inches='tight')
        plt.close()
        cluster_labels[cluster_id] = ', '.join(list(wordcloud.words_.keys())[:5])
    
    df['Cluster_Label'] = df['Product_Cluster'].map(cluster_labels)
    
    # PCA Visualization
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(tfidf_matrix.toarray())
    df['PCA1'] = reduced_features[:,0]
    df['PCA2'] = reduced_features[:,1]
    
    plt.figure(figsize=(14,8))
    sns.scatterplot(x='PCA1', y='PCA2', hue='Cluster_Label', 
                   data=df, palette='tab10', alpha=0.7)
    plt.title('Product Cluster Visualization')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.savefig('04 visualizations/product_clusters_pca.png', bbox_inches='tight', dpi=300)
    plt.close()
    
    return df

#----------------------------------------------------------------------------------------------------------------------------
# Task 3: Association Rule Mining
#----------------------------------------------------------------------------------------------------------------------------
def generate_association_rules(df):
    """Generate association rules using FP-growth"""
    print("\n=== Market Basket Analysis ===")
    
    # Create basket matrix
    basket = (df.groupby(['InvoiceNo', 'Cluster_Label'])['Quantity']
             .sum().unstack().fillna(0)
             .applymap(lambda x: 1 if x > 0 else 0))

    # FP-growth implementation
    frequent_itemsets = fpgrowth(basket, min_support=0.03, use_colnames=True)
    
    if not frequent_itemsets.empty:
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
        rules = rules.sort_values('confidence', ascending=False)
        
        # Rule visualizations
        plt.figure(figsize=(12,6))
        top_rules = rules.head(15).sort_values('confidence', ascending=True)
        plt.barh(y=range(len(top_rules)), width=top_rules['confidence'], 
                color=sns.color_palette("viridis", len(top_rules)))
        plt.yticks(range(len(top_rules)), [f"{a} → {c}" for a,c in zip(top_rules['antecedents'], top_rules['consequents'])])
        plt.title('Top 15 Association Rules by Confidence')
        plt.savefig('04 visualizations/top_rules.png', dpi=300)
        plt.close()

        # Support vs Lift
        plt.figure(figsize=(10,6))
        sns.scatterplot(data=rules, x='support', y='lift', hue='confidence',
                       palette='viridis', sizes=(20, 200))
        plt.title('Support vs Lift')
        plt.savefig('04 visualizations/support_lift_confidence.png')
        plt.close()

        # Network graph
        G = nx.DiGraph()
        top_rules_graph = rules.sort_values('lift', ascending=False).head(30)
        for _, row in top_rules_graph.iterrows():
            antecedent = list(row['antecedents'])
            consequent = list(row['consequents'])
            G.add_edge(', '.join(antecedent), ', '.join(consequent), 
                      label=f"Conf: {row['confidence']:.2f}")
        
        plt.figure(figsize=(15,10))
        pos = nx.spring_layout(G)
        nx.draw(G, pos, with_labels=True, node_size=2000, 
               font_size=8, arrowsize=20)
        plt.title('Association Rule Network')
        plt.savefig('04 visualizations/rules_network.png', dpi=300)
        plt.close()
        
        return rules
    return pd.DataFrame()

#----------------------------------------------------------------------------------------------------------------------------
# Task 4: Customer Segmentation
#----------------------------------------------------------------------------------------------------------------------------
def perform_customer_segmentation(df):
    """Segment customers using purchasing behavior"""
    print("\n=== Customer Segmentation ===")
    
    customer_features = df.groupby('CustomerID').agg({
        'UnitPrice': 'mean',
        'Quantity': 'sum',
        'InvoiceNo': 'nunique'
    }).reset_index()
    
    scaler = StandardScaler()
    processed_data = scaler.fit_transform(customer_features[['UnitPrice', 'Quantity', 'InvoiceNo']])
    
    kmeans = KMeans(n_clusters=4, random_state=2040, n_init=20)
    customer_features['Segment'] = kmeans.fit_predict(processed_data)
    
    # PCA Visualization
    pca = PCA(n_components=2)
    reduced_data = pca.fit_transform(processed_data)
    customer_features[['PCA1', 'PCA2']] = reduced_data
    
    plt.figure(figsize=(12,8))
    sns.scatterplot(x='PCA1', y='PCA2', hue='Segment',
                   data=customer_features, palette='Set2', s=100)
    plt.title('Customer Segmentation')
    plt.savefig('04 visualizations/customer_segments.png', dpi=300)
    plt.close()
    
    return customer_features

#----------------------------------------------------------------------------------------------------------------------------
# Task 5: Report Generation
#----------------------------------------------------------------------------------------------------------------------------
class RetailReportGenerator:
    def __init__(self):
        self.doc = Document()
        self._add_title_page()
    
    def _add_title_page(self):
        self.doc.add_heading('Retail Analytics Report', 0)
        self.doc.add_paragraph(f"Generated: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}")
        self.doc.add_page_break()
    
    def add_section(self, title, content, images=None):
        self.doc.add_heading(title, level=1)
        self.doc.add_paragraph(content)
        if images:
            for img_path in images:
                if os.path.exists(img_path):
                    self.doc.add_picture(img_path, width=Inches(6))
        self.doc.add_page_break()
    
    def generate_report(self, df, rules, segments):
        # Data Overview
        self.add_section(
            "Data Overview",
            f"Analyzed {len(df):,} transactions with {df['StockCode'].nunique():,} unique products",
            ['04 visualizations/clustering_metrics.png']
        )
        
        # Product Analysis
        cluster_images = [f'04 visualizations/cluster_{i}_keywords.png' for i in range(6)]
        self.add_section(
            "Product Clustering",
            "Product groups identified through text analysis:",
            cluster_images + ['04 visualizations/product_clusters_pca.png']
        )
        
        # Market Basket Analysis
        if not rules.empty:
            self.add_section(
                "Association Rules",
                "Strong product associations identified:",
                ['04 visualizations/top_rules.png',
                 '04 visualizations/support_lift_confidence.png',
                 '04 visualizations/rules_network.png']
            )
        
        # Customer Analysis
        self.add_section(
            "Customer Segmentation",
            "Customer groups based on purchasing behavior:",
            ['04 visualizations/customer_segments.png']
        )
        
        self.doc.save('comprehensive_retail_report.docx')

#----------------------------------------------------------------------------------------------------------------------------
# Main Execution
#----------------------------------------------------------------------------------------------------------------------------
if __name__ == "__main__":
    # Data pipeline
    df = load_and_clean_data()
    df = cluster_products(df)
    
    # Analytics pipeline
    rules = generate_association_rules(df)
    customer_segments = perform_customer_segmentation(df)
    
    # Reporting
    report = RetailReportGenerator()
    report.generate_report(df, rules, customer_segments)
    
    # Save outputs
    df.to_csv('clustered_products.csv', index=False)
    customer_segments.to_csv('customer_segments.csv', index=False)
    
    print("\n✅ Analysis Complete")
    print("Generated Files:")
    print("- comprehensive_retail_report.docx")
    print("- 04 visualizations/ [All analysis charts]")
    print("- clustered_products.csv")
    print("- customer_segments.csv")


=== Data Cleaning ===
Initial records: 541910

=== Product Clustering ===


In [ ]:
#----------------------------------------------------------------------------------------------------------------------------
#Task B: Make decision tree from the clustered descriptions
#----------------------------------------------------------------------------------------------------------------------------
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

#----------------------------------------------------------------------------------------------------------------------------
# Setup
#----------------------------------------------------------------------------------------------------------------------------
os.makedirs('04 visualizations', exist_ok=True)

#----------------------------------------------------------------------------------------------------------------------------
# 1. Load and prepare data
#----------------------------------------------------------------------------------------------------------------------------
df = pd.read_excel("description_retail.xlsx")
df = df.dropna(subset=['Description'])
df['Description'] = df['Description'].astype(str).str.lower().str.strip()

#----------------------------------------------------------------------------------------------------------------------------
#Task 2. TF-IDF Vectorization
#----------------------------------------------------------------------------------------------------------------------------
tfidf = TfidfVectorizer(max_df=0.8, min_df=5, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Description'])

#----------------------------------------------------------------------------------------------------------------------------
#Task3. KMeans Clustering
#----------------------------------------------------------------------------------------------------------------------------
k = 6
model = KMeans(n_clusters=k, random_state=2040)
df['cluster'] = model.fit_predict(tfidf_matrix)

#----------------------------------------------------------------------------------------------------------------------------
#Task 4. Labeling Clusters
#----------------------------------------------------------------------------------------------------------------------------
cluster_labels = {
    0: "Gift Sets & Stationery",
    1: "Accessories & Decor",
    2: "Seasonal Holiday Items",
    3: "Bags & Gifting Items",
    4: "Candle Holders & Lighting",
    5: "Boxes & Storage Items"
}
df['Cluster_Label'] = df['cluster'].map(cluster_labels)

#----------------------------------------------------------------------------------------------------------------------------
#Task 5. Dimensionality reduction for plot
#----------------------------------------------------------------------------------------------------------------------------
pca = PCA(n_components=2)
reduced = pca.fit_transform(tfidf_matrix.toarray())
df['PCA1'] = reduced[:, 0]
df['PCA2'] = reduced[:, 1]

#----------------------------------------------------------------------------------------------------------------------------
#Task 6. Visualization
#----------------------------------------------------------------------------------------------------------------------------
plt.figure(figsize=(12, 8))
sns.scatterplot(
    x='PCA1', y='PCA2',
    hue='Cluster_Label',
    data=df,
    palette='tab10',
    alpha=0.7
)
plt.title("Item description clusters")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig("04 visualizations/item_clusters_named.png")
plt.close()

#----------------------------------------------------------------------------------------------------------------------------
#Task 7. Load retail dataset and merge
#----------------------------------------------------------------------------------------------------------------------------
retail_df = pd.read_excel("retail_dataset.xlsx")
retail_df = retail_df.rename(columns={
    'Invoice': 'InvoiceNo',
    'Price': 'UnitPrice',
    'Customer ID': 'CustomerID'
})
retail_df['Description'] = retail_df['Description'].astype(str).str.lower().str.strip()
merged_df = pd.merge(retail_df, df[['Description', 'Cluster_Label']], on='Description', how='left')

#----------------------------------------------------------------------------------------------------------------------------
#Task 8. Create price category and ensure quantity is positive
#----------------------------------------------------------------------------------------------------------------------------
merged_df['Quantity'] = merged_df['Quantity'].abs()
merged_df['Price_Cat'] = pd.cut(merged_df['UnitPrice'], 
                                bins=[-float('inf'), 5, 10, 50, float('inf')],
                                labels=['Budget', 'Mid-Range', 'Premium', 'Luxury'])

#----------------------------------------------------------------------------------------------------------------------------
#Task 9. Decision tree to predict price catgory
#----------------------------------------------------------------------------------------------------------------------------
merged_df = merged_df.dropna(subset=['Cluster_Label', 'Price_Cat', 'Country'])

le = LabelEncoder()
X = merged_df[['Country', 'Cluster_Label', 'Quantity']].copy()
X['Country'] = le.fit_transform(X['Country'])
X['Cluster_Label'] = le.fit_transform(X['Cluster_Label'])
y = le.fit_transform(merged_df['Price_Cat'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2040)
clf = DecisionTreeClassifier(max_depth=3, random_state=2040)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

#----------------------------------------------------------------------------------------------------------------------------
#Task 10. Save decision tree visualization
#----------------------------------------------------------------------------------------------------------------------------
plt.figure(figsize=(20, 10))
plot_tree(clf, filled=True,
          feature_names=X.columns.tolist(),
          class_names=le.classes_)
plt.title(f'Price Category Prediction Tree (Accuracy: {accuracy:.2%})')
plt.savefig('04 visualizations/decision_tree_price_cat.png', bbox_inches='tight')
plt.close()

#----------------------------------------------------------------------------------------------------------------------------
#Task 11. Save final merged dataset
#----------------------------------------------------------------------------------------------------------------------------
merged_df.to_excel("04 visualizations/retail_with_clusters.xlsx", index=False)

accuracy
